In [1]:
import sys
sys.path.append("../")
import pandas as pd
import scraping.forexFactory_com as ff

import pandas as pd
from dateutil import parser
from sklearn.preprocessing import StandardScaler

import re
import numpy as np
pd.set_option('display.max_rows', None)  # Display all rows
pd.set_option('display.max_columns', None)
import re
import numpy as np
import pandas as pd

from sklearn.impute import KNNImputer
from sklearn.preprocessing import LabelEncoder


ModuleNotFoundError: No module named 'scraping.forexFactory_com'

In [ ]:
forexfox = ff.get_monthly_data("nov.2024").copy()

forexfox_an = forexfox.copy()
forexfox.head(75)



In [ ]:
forexfox_an.head(75)


In [ ]:
def calculate_rate_of_change(self):
        """Calculate rate of change between actual, previous, and forecast values."""
        forexfox_an['rate_change_actual_previous'] = (
            (self.forexfox_an['actual'] - forexfox_an['previous']) /forexfox_an['previous']
        )
        forexfox_an['rate_change_actual_forecast'] = (
            (forexfox_an['actual'] - forexfox_an['forecast']) / forexfox_an['forecast']
        )
        return forexfox_an[['Date', 'rate_change_actual_previous', 'rate_change_actual_forecast']]


In [ ]:
date_time_list = []
for index, row in forexfox_an.iterrows():
    # Check if 'Time' is set to "All Day"
    time_value = row['Time'] if row['Time'] != 'All Day' else '12:00am'
    date_string = f"{row['Date']} {time_value}"
    
    try:
        parsed_date = parser.parse(date_string)
    except parser.ParserError:
        print(f"Skipping invalid date format: {date_string}")
        parsed_date = None
    
    date_time_list.append(parsed_date)

# Assign the parsed date-time values to a new 'DateTime' column
forexfox_an['DateTime'] = date_time_list





In [ ]:
forexfox_an.head()

In [ ]:
forexfox_an.head()
forexfox_norma =forexfox_an.copy()


In [ ]:
# Step 3: Feature Engineering - Extract useful information from 'DateTime'
forexfox_an['DayOfWeek'] = forexfox_an['DateTime'].dt.dayofweek
forexfox_an['Hour'] = forexfox_an['DateTime'].dt.hour

# Step 4: Drop original 'Date', 'Time', and 'DateTime' columns
forexfox_an = forexfox_an.drop(['DateTime'], axis=1)
forexfox_an = forexfox_an.drop(['Time'], axis=1)
forexfox_an = forexfox_an.drop(['Date'], axis=1)

# # Drop rows where 'DateTime' could not be parsed
# forexfox.dropna(subset=['DateTime'], inplace=True)


In [ ]:


# List of symbols to remove
SYMBOLS_TO_REMOVE = [r'%', r'M', r'¥', r'\$', r'€', r'C\$', r'K', r'B', r'A', r'\s', r',']
pattern = '|'.join(SYMBOLS_TO_REMOVE)

# Define the standalone function
def remove_symbols(column):
    """Remove specified symbols from a column, handle multi-values, and convert to float."""
    cleaned_column = []
    
    for value in column:
        # Convert the value to a string and remove symbols using regex
        cleaned_value = re.sub(pattern, '', str(value))
        
        # Handle cases with multiple values separated by '|'
        if '|' in cleaned_value:
            # Split by '|' and take the first part
            cleaned_value = cleaned_value.split('|')[0]
        
        # Replace empty strings with NaN
        if cleaned_value == '':
            cleaned_column.append(np.nan)
        else:
            try:
                cleaned_column.append(float(cleaned_value))
            except ValueError:
                # If conversion to float fails, set as NaN
                cleaned_column.append(np.nan)
    
    # Return the cleaned values as a Pandas Series
    return pd.Series(cleaned_column)



# Apply `remove_symbols` to the specified columns
forexfox_an['Actual'] = remove_symbols(forexfox_an['Actual'])
forexfox_an['Previous'] = remove_symbols(forexfox_an['Previous'])
forexfox_an['Forecast'] = remove_symbols(forexfox_an['Forecast'])
# Keep only rows where at least one of 'Actual', 'Forecast', or 'Previous' is not NaN
forexfox_an.dropna(subset=['Actual', 'Forecast', 'Previous'], how='all', inplace=True)


# Display the cleaned DataFrame
print(forexfox_an.head(40))




In [ ]:
le_currency = LabelEncoder()
le_event = LabelEncoder()
forexfox_an['Currency'] = le_currency.fit_transform(forexfox_an['Currency'])
forexfox_an['Event'] = le_event.fit_transform(forexfox_an['Event'])

In [ ]:
imputer = KNNImputer(n_neighbors=3)
forexfox_an_imputed = pd.DataFrame(imputer.fit_transform(forexfox_an), columns=forexfox_an.columns)

print(forexfox_an_imputed)

In [ ]:
reference_date = pd.to_datetime("2024-11-01")

def reconstruct_datetime(row):
    # Calculate the actual date using reference_date and 'DayOfWeek'
    adjusted_date = reference_date + pd.to_timedelta((row['DayOfWeek'] - reference_date.weekday()) % 7, unit='D')
    return pd.Timestamp(adjusted_date.year, adjusted_date.month, adjusted_date.day, int(row['Hour']), 0)

# Apply the function to reconstruct the DateTime column
forexfox_an_imputed['DateTime'] = forexfox_an_imputed.apply(reconstruct_datetime, axis=1)

# Step 9: Drop feature-engineered columns if they are not needed
forexfox_an_imputed = forexfox_an_imputed.drop(['DayOfWeek', 'Hour'], axis=1)

# Display the cleaned and imputed DataFrame
print(forexfox_an_imputed.head(40))

